<a href="https://colab.research.google.com/github/sivannavis/NLP-for-human-rights/blob/main/Final_demo_WordEmbed_for_10k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embedding for 10Ks from 2016 to 2021

In [ ]:
import pandas as pd
import itertools
import spacy
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

# import chart_studio
# import chart_studio.plotly as py
# import chart_studio.tools as tls
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

from tqdm.notebook import tqdm_notebook
from tqdm import  tqdm
tqdm_notebook.pandas(desc="progress bar")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk.corpus import stopwords
import nltk.data
import logging
from gensim.models import Word2Vec
from gensim.models import fasttext
import pickle


## read in 10k data for a all years
Takes 5min to read from 2016 to 2021.
Takes about 2 min for each csv to drop dupes and save to file.

In [ ]:
# navigating to the dataset directory in drive
%cd "~/sd3507@columbia.edu - Google Drive/My Drive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed"
!pwd
!ls

/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed
/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed
10ks_para                   complete_10ks.csv
10ks_para.parts             labeled_paragraphed.csv
10ks_para_ner.parts         log_0515-1505.txt
10ks_para_sics              log_0515-1535.txt
2020_DEI_10ks               superseded
bmi_labeled_paragraphed.csv tri_gram_grid_search
complete_10ks


In [ ]:
dir_10ks = '10ks_para_sics/'
file_name_prefix_10ks = dir_10ks + '10ks_para_sics_'



In [ ]:
# read in dataframe from 2016 to 2022

selected_year_10ks_sample_dir = f'complete_10ks' # os.mkdir(selected_year_10ks_sample_dir)

for year in range(2016,2022):
  print(year)
  exec('df_{}_10ks = pd.DataFrame()'.format(year))
  current_df = pd.read_csv(file_name_prefix_10ks + str(year) + '.csv')
  

  # drop duplicates
  current_df['10K_id'] = current_df.apply(lambda row: row['id'].rsplit('_', 1)[0], axis=1)
  current_df_deduped = current_df.drop_duplicates(subset=['10K_id', 'text'], keep='first')
  current_df_deduped.drop(['10K_id'], inplace=True, axis=1)
  globals()['df_'+str(year)+'_10ks'] = current_df_deduped

  # save to files
  current_df_deduped.to_csv(f'{selected_year_10ks_sample_dir}/df_{year}_10ks_deduped.csv', index=False)

2020
2021


In [ ]:
# working directory
%cd "complete_10ks"
!pwd

# readin all the files (takes about 3 min from 2016 to 2022)
for year in range(2016,2022):
  print(year)
  exec('df_{}_10ks = pd.DataFrame()'.format(year))
  globals()['df_'+str(year)+'_10ks'] = pd.read_csv(f'df_{year}_10ks_deduped.csv')

/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks
/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks
2016
2017
2018
2019
2020
2021


In [ ]:
# put deduped df together

full_10ks = [df_2016_10ks, df_2017_10ks, df_2018_10ks, df_2019_10ks, df_2020_10ks, df_2021_10ks]
df_10ks_16_21 = df_2016_10ks
df_10ks_16_21 = pd.concat(full_10ks)
df_10ks_16_21

,id,ticker_display,primary_industry_id,text
0,2016Q1_SC_10-K_20151231_0001580608-16-000185.t...,SC,FN-CF,Indicate by check mark whether the registrant ...
1,2016Q1_SC_10-K_20151231_0001580608-16-000185.t...,SC,FN-CF,Indicate by check mark whether the registrant ...
2,2016Q1_SC_10-K_20151231_0001580608-16-000185.t...,SC,FN-CF,Indicate by check mark if disclosure of delinq...
3,2016Q1_SC_10-K_20151231_0001580608-16-000185.t...,SC,FN-CF,Portions of the registrant's definitive proxy ...
4,2016Q1_SC_10-K_20151231_0001580608-16-000185.t...,SC,FN-CF,This Annual Report on Form 10-K contains forwa...
...,...,...,...,...
1976643,2021Q1_BXG_10-K_20201231_0000778946-21-000012....,BXG,SV-HL,NBA Receivables Facility. Bluegreen/Big Ce...
1976644,2021Q1_BXG_10-K_20201231_0000778946-21-000012....,BXG,SV-HL,efforts to renew or replace credit facilities ...
1976645,2021Q1_BXG_10-K_20201231_0000778946-21-000012....,BXG,SV-HL,REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOU...
1976646,2021Q1_BXG_10-K_20201231_0000778946-21-000012....,BXG,SV-HL,interest due under the Liberty Bank Facility a...


In [ ]:
# saving complete csv takes 5 minutes.
print(df_10ks_16_21.shape)
df_10ks_16_21.to_csv('16_21_10ks.csv.gz', compression='gzip', index=False)


(11443419, 4)


## data preprocessing

### tokenizers(unigram)
text to sentense runtime: ~4s for 3k samples

In [ ]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

# stopwords_exp = stopwords.words("english") + ['e','d']
my_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
       "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
       'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
       'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
       'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
       'this', 'that', "that'll", 'these', 'those', 'am',
       'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
       'but', 'if', 'because', 's', 't',
       'don', "don't", 'should', "should've", 'now', 'd', 'll',
       'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn',
       "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
       'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma',
       'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
       'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't",
       'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"] + ['e', 'd', 'b', 'k','p']

def text_to_wordlist( text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(text).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(my_stopwords)
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [ ]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("popular")   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# load stopwords
nltk.download("stopwords")

# Define a function to split a review into parsed sentences
def text_to_sentences( text, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( text_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words, so this returns a list of lists
    return sentences

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/sivanding/nltk_data...
[nltk_data]    |   Package movie_

In [ ]:
# tokenizing

sentences = []  # Initialize an empty list of sentences
nltk.download("stopwords")
from nltk.corpus import stopwords
import pickle
import time
import timeout_decorator 


# time out helper code: limit runtime to 1s, skip otherwise
# because normally each row takes way less than 1 second
@timeout_decorator.timeout(1) 
def tts(data): 
    #loop code goes here
    return text_to_sentences(data, tokenizer, True) 

print ("Parsing sentences from training set")

for year in range(2016, 2022):
  
  sentences = []
  work_df = globals()['df_'+str(year)+'_10ks']
  globals()["skip_"+str(year)]  = []
  
  for index in range(len(work_df)):
    # print(text)
    print("now on row {} in year {}".format(index, year))
    text = work_df['text'][index]
    try:
      cur_sentence = tts(text)
      sentences += cur_sentence
    except Exception: 
      print("skipped {}".format(index))
      globals()["skip_"+str(year)].append(index)
      continue

  with open('unigram_tokenizers/unigram_{}.pkl'.format(year), 'wb') as f:
    pickle.dump(sentences, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# text output wrapper
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Putting together complete sentences tokenizers

In [ ]:
# reading tokens takes 1 hour in local runtime.......
%cd "complete_10ks/unigram_tokenizers"

In [ ]:

import os
comp_sentences = []

for year in range(2016, 2022):
  with open("unigram_{}.pkl".format(year), 'rb') as f:
    globals()['sentences_'+str(year)] = pickle.load(f)
  comp_sentences += globals()['sentences_'+str(year)]
  print(str(year) + "done...")

/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks/unigram_tokenizers
2016done...
2017done...
2018done...
2019done...
2020done...
2021done...


In [ ]:
import gzip
with gzip.open('comp_sentences.gz', 'wb') as f:
  # for sentence in sentences:
  pickle.dump(comp_sentences, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with gzip.open('comp_sentences.gz', 'rb') as f:
  # for sentence in sentences:
  comp_sentences = pickle.load(f)

In [ ]:
# check sentences
print(len(comp_sentences))
print(np.array(comp_sentences[:10]))


45470057
[list(['indicate', 'by', 'check', 'mark', 'whether', 'registrant', 'has', 'filed', 'all', 'reports', 'required', 'to', 'be', 'filed', 'by', 'section', 'or', 'of', 'securities', 'exchange', 'act', 'of', 'during', 'preceding', 'months', 'or', 'for', 'such', 'shorter', 'period', 'registrant', 'was', 'required', 'to', 'file', 'such', 'reports', 'and', 'has', 'been', 'subject', 'to', 'such', 'filing', 'requirements', 'for', 'past', 'days'])
 list(['yes', 'no'])
 list(['indicate', 'by', 'check', 'mark', 'whether', 'registrant', 'has', 'submitted', 'electronically', 'and', 'posted', 'on', 'corporate', 'web', 'site', 'any', 'every', 'interactive', 'data', 'file', 'required', 'to', 'be', 'submitted', 'and', 'posted', 'pursuant', 'to', 'rule', 'of', 'regulation', 'st', 'section', 'of', 'chapter', 'during', 'preceding', 'months', 'or', 'for', 'such', 'shorter', 'period', 'registrant', 'was', 'required', 'to', 'submit', 'and', 'post', 'such', 'files'])
 list(['yes', 'no'])
 list(['indicat

/var/folders/s3/cd0v0kp570g2sxvzwfq2vxdc0000gn/T/ipykernel_2540/4126310492.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(comp_sentences[:10]))


### n-gram pipelines: phrases within tokenizing (**with Phraser**)

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:

def build_phrases(sentences, min_count, threshold):
    phrases = Phrases(sentences,
                      min_count=min_count,
                      threshold=threshold,
                      progress_per=1000)
    return Phraser(phrases)


In [ ]:
# build phrase model
comp_bi_phrases_model = build_phrases(comp_sentences, 5, 100)

# save bi-gram phrase model
comp_bi_phrases_model.save('comp_bi_phrases_model.txt')
comp_bi_phrases_model= Phraser.load('comp_bi_phrases_model.txt')

In [ ]:
# bi-gram extraction
def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])

# bi-gram concatenation
def sentences_to_bi_grams(n_grams, tokenized_sentences, output_file_name): 
    # with open(input_file_name, 'r') as input_file_pointer: 
        with open(output_file_name, 'w+') as out_file: 
            for tokenized_sentence in tokenized_sentences: 
                # clean_sentence = clean_sentence(sentence) 
                # tokenized_sentence = tokenize(cleaned_sentence) 
                parsed_sentence = sentence_to_bi_grams(n_grams, tokenized_sentence) 
                out_file.write(parsed_sentence + '\n')

In [ ]:
sentences_to_bi_grams(comp_bi_phrases_model, comp_sentences, "comp_bigram.txt")

In [ ]:
# build bi-gram form tokenizers (53 min)
comp_bi_sentences = []
for bi_sentence in open("comp_bigram.txt").readlines():
  comp_bi_sentences.append(bi_sentence.split())
print(comp_bi_sentences[:10])

[['indicate', 'by', 'check', 'mark', 'whether', 'registrant', 'has', 'filed', 'all', 'reports', 'required', 'to', 'be', 'filed', 'by', 'section', 'or', 'of', 'securities', 'exchange', 'act', 'of', 'during', 'preceding', 'months', 'or', 'for', 'such', 'shorter', 'period', 'registrant', 'was', 'required', 'to', 'file', 'such', 'reports', 'and', 'has', 'been', 'subject', 'to', 'such', 'filing', 'requirements', 'for', 'past', 'days'], ['yes', 'no'], ['indicate', 'by', 'check', 'mark', 'whether', 'registrant', 'has', 'submitted', 'electronically', 'and', 'posted', 'on', 'corporate', 'web', 'site', 'any', 'every', 'interactive', 'data', 'file', 'required', 'to', 'be', 'submitted', 'and', 'posted', 'pursuant', 'to', 'rule', 'of', 'regulation', 'st', 'section', 'of', 'chapter', 'during', 'preceding', 'months', 'or', 'for', 'such', 'shorter', 'period', 'registrant', 'was', 'required', 'to', 'submit', 'and', 'post', 'such', 'files'], ['yes', 'no'], ['indicate', 'by', 'check', 'mark', 'disclosure

In [ ]:
comp_bi_phrases_model= Phraser.load('comp_bi_phrases_model.txt')
print(len(comp_bi_phrases_model.phrasegrams.keys()))

140487


In [ ]:
# build trigram and four-gram phrase model
comp_tri_phrases_model = build_phrases(comp_bi_sentences, 5, 10)

# save tri-gram and four-gram phrase model
comp_tri_phrases_model.save('comp_tri_phrases_model.txt')


### building tri-gram and four-gram in similar approach


In [ ]:
# tri-gram (16min to generate and 1h to read)
comp_tri_phrases_model= Phraser.load('comp_tri_phrases_model.txt')
print(len(comp_tri_phrases_model.phrasegrams.keys()))


2022-05-24 08:48:58,837 : INFO : loading FrozenPhrases object from comp_tri_phrases_model.txt
2022-05-24 08:48:59,094 : INFO : FrozenPhrases lifecycle event {'fname': 'comp_tri_phrases_model.txt', 'datetime': '2022-05-24T08:48:59.094188', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}
2022-05-24 08:48:59,095 : WARNING : older version of FrozenPhrases loaded without corpus_word_count
2022-05-24 08:48:59,096 : WARNING : setting corpus_word_count to 0, do not use it in your scoring function


290623


In [ ]:
sentences_to_bi_grams(comp_tri_phrases_model, comp_bi_sentences, "comp_trigram.txt")

In [ ]:
tri_sentences = []
for tri_sentence in open("comp_sample_trigram.txt").readlines():
    tri_sentences.append(tri_sentence.split())

In [ ]:
uni_gram = []
bi_grams = []
tri_grams = []
four_grams = []

  
# Manually delete four-gram and count grams
for sentence_i in range(len(tri_sentences)):
  for index in range(len(tri_sentences[sentence_i])-1, 0, -1):
    if tri_sentences[sentence_i][index].count("_") > 2:
      four_grams.append(tri_sentences[sentence_i][index])
      # del tri_sentences[sentence_i][index]
    elif tri_sentences[sentence_i][index].count("_") == 1:
      bi_grams.append(tri_sentences[sentence_i][index])
    elif tri_sentences[sentence_i][index].count("_") == 2:
      tri_grams.append(tri_sentences[sentence_i][index])
    else:
      uni_gram.append(tri_sentences[sentence_i][index])

uni_gram = set(uni_gram)
bi_grams = set(bi_grams)
tri_grams =set(tri_grams)
four_grams =set(four_grams)

# preview grams
print(tri_sentences[:10])
print(uni_gram[:10])
print(bi_grams[:10])
print(tri_grams[:10])
print(four_grams[:10])

# see sizes
# print(sum( [len(listElem) for listElem in tri_sentences]))
print(len(set([tuple(t) for t in sentences])))
print(len(uni_gram))
print(len(bi_grams))
print(len(tri_grams))
print(len(four_grams))
# print(len(tri_phrases_model.phrasegrams.keys()))
# print(tri_phrases_model.phrasegrams.keys())


## Training Word2Vec model with tri-gram tokenizers
lower min_word_count because of smaller vocab

runtime: ~1s

complete model: 31 min

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 150    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count - this limits vocabulary size              
num_workers = 4       # Number of threads to run in parallel
context = 15          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
from gensim.models import Word2Vec
print("Training model...")

# Using default architecture: skip-gram
tri_model = word2vec.Word2Vec(tri_sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, epochs = 2)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
tri_model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
tri_model_name = "comp_150features_5minwords_15context_trigram"
tri_model.save(tri_model_name)

2022-05-24 02:56:17,117 : INFO : collecting all words and their counts
2022-05-24 02:56:17,123 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-24 02:56:17,228 : INFO : PROGRESS: at sentence #10000, processed 243326 words, keeping 8981 word types


Training model...


Streaming output truncated to the last 5000 lines.
2022-05-24 02:57:27,400 : INFO : PROGRESS: at sentence #9870000, processed 226904588 words, keeping 300631 word types
2022-05-24 02:57:27,488 : INFO : PROGRESS: at sentence #9880000, processed 227135217 words, keeping 300810 word types
2022-05-24 02:57:27,553 : INFO : PROGRESS: at sentence #9890000, processed 227373479 words, keeping 300971 word types
2022-05-24 02:57:27,623 : INFO : PROGRESS: at sentence #9900000, processed 227594982 words, keeping 301029 word types
2022-05-24 02:57:27,726 : INFO : PROGRESS: at sentence #9910000, processed 227825262 words, keeping 301053 word types
2022-05-24 02:57:27,811 : INFO : PROGRESS: at sentence #9920000, processed 228073080 words, keeping 301061 word types
2022-05-24 02:57:27,882 : INFO : PROGRESS: at sentence #9930000, processed 228307608 words, keeping 301097 word types
2022-05-24 02:57:27,954 : INFO : PROGRESS: at sentence #9940000, processed 228532807 words, keeping 301125 word types
2022-

## Tests for tri-gram model with keywords

In [ ]:
# navigating to the dataset directory in drive
%cd "~/sd3507@columbia.edu - Google Drive/My Drive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks/unigram_tokenizers"
!pwd
!ls

/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks/unigram_tokenizers
/Volumes/GoogleDrive-110853384825063897304/.shortcut-targets-by-id/1OPzVNu0CgKsi5tBwlMJVXZTu1EWX9F6C/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/complete_10ks/unigram_tokenizers
comp_150features_5minwords_15context_trigram
comp_150features_5minwords_15context_trigram.syn1neg.npy
comp_150features_5minwords_15context_trigram.wv.vectors.npy
comp_bi_phrases_model.txt
comp_bigram.txt
comp_sentences.gz
comp_tri_phrases_model.txt
comp_trigram.txt
unigram_2016.pkl
unigram_2017.pkl
unigram_2018.pkl
unigram_2019.pkl
unigram_2020.pkl
unigram_2021.pkl


In [ ]:
# load model
tri_model_name = "comp_150features_5minwords_15context_trigram"
tri_model = Word2Vec.load(tri_model_name)

In [ ]:
# vector comparison
tri_words = tri_model.wv.index_to_key
print(len(tri_words))
print(np.array(tri_words[:2000]))

416458
['of' 'and' 'to' ... 'mutual' 'repaid' 'structured']


In [ ]:
from posixpath import pardir
# check similarity

risk_terms=  ['strike', 'sit_in', 'operational_disruption', 'protest', 'injury', 
              'operational_disruption', 'operating_cost', 'delay', 'disruption',
              'lawsuit', 'litigation', 'impoundment', 'detain','penalt', 'sanction', 'court', 'consent_decree',
              'brand_damage', 'monetary_damage','brand_reputation', 'brand_recognition','social_license','decreased_trust','decreased_innovation','lost_opportunity', 'resign',
              'divest', 'restitution', 'conciliation_agreement',
              'modern_slavery', 'debt_bondage', 'human_traffic',
              'alleg', 'accus', 'exploit', 'publicly_expose', 'investigat', 'enforcement', 'security_force', 'inspection', 'inspector',
              'scandal', 'government_action', 'share_price', 'share_value']

practice_terms = ['wage','wage_theft','living_wage',
                  'precarity', 'precarious_work','gig_work','alternative_work','alternate_work','contingent_work','informal_work','casual_work','hazardous_work',
                  'confinement', 'document_retention','restriction_of_movement','delayed_wage','pay_manipulation','punishment', 'poor food','deprivation','unpaid_wage', 'delayed_payment','wage_violation',
                  'freedom_of_association','collective_bargaining', 'work_stoppage','hotline',
                  'code_of_conduct', 'due_diligence','ethical_recruit', 'handbook','social_audit','equal_benefits','transparency', 'traceability', 'visibility', 'accessibility','timely_payments','union', 'worker_committee','empower', 'accommodation',
                  'outsource', 'subcontracting','program', 'initiative', 'training','exempt', 'recruit', 'promotion', 'arbitration','mentorship', 'affirmative_action',
                   'order_delay','lead_time','unplanned_shipment','corruption', 'quota_system', 'delayed_payment','weak_governance', 'wage_violation','informal_supply_chain', 'last-minute_order_modification', 'unfair_timing_demand', 'pricing_pressure','poor_forecasting', 'irresponsible_exit']

# new demo test
practice = ['hiring', 'recruitment', 'employee_training_protect_whistleblowers', 'retain_workers', 'attract_talent', 'promotion', 'provide_religious_accommodations', 'offer_flexible_work', 'equal_pay']
practice = ['hiring', 'recruitment', 'training', 'whistleblower', 'retention', 'talent_attraction', 'promotion', 'accommodation','accommodations', 'arbitration', 'remuneration']

def check_occ(terms, vocab):
  coterms = []
  for term in terms:
    if term in vocab:
      coterms.append(term)
  return coterms



In [ ]:
risk_coterms = check_occ(risk_terms, tri_words)
practice_coterms = check_occ(practice, tri_words)
print("risk:", len(risk_coterms), "practice:", len(practice_coterms))


risk: 21 practice: 11


In [ ]:
# most similar
print("Risks:")
for term in risk_coterms:
  print("most_similar", term, tri_model.wv.most_similar(term, topn = 3))

  # different distances
  # print("most_similar_cosmul", term, tri_model.wv.most_similar_cosmul(term, topn = 5))
  # print("centroid", term, tri_model.wv.rank_by_centrality(term))
  # print("similar_by_key", term, tri_model.wv.similar_by_word(term))

print("Practice:")
for term in practice_coterms:
  print(term, tri_model.wv.most_similar(term, topn = 3))

Risks:
most_similar strike [('waterhouse', 0.5382134914398193), ('waterhouse_llp', 0.5271006226539612), ('exercise', 0.5001725554466248)]
most_similar protest [('protests', 0.7015029191970825), ('protested', 0.6375338435173035), ('usitc', 0.524922788143158)]
most_similar injury [('injuries', 0.7647931575775146), ('personal_injury', 0.6362467408180237), ('damage', 0.6278994679450989)]
most_similar delay [('delays', 0.6547819972038269), ('delayed', 0.6080998182296753), ('halt', 0.5982756614685059)]
most_similar disruption [('disruptions', 0.8936829566955566), ('interruptions', 0.7992265224456787), ('interruption', 0.7486926317214966)]
most_similar lawsuit [('complaint', 0.7108051180839539), ('suit', 0.7042682766914368), ('lawsuits', 0.6651405096054077)]
most_similar litigation [('lawsuits', 0.7656179070472717), ('litigations', 0.7093724012374878), ('proceedings', 0.6954681873321533)]
most_similar impoundment [('ponds', 0.6440322399139404), ('quarry', 0.619916558265686), ('pits', 0.606535

In [ ]:
# testing example for current model
tri_test = []
for term in risk_coterms+practice_coterms:
  tri_test.append(tri_model.wv.most_similar(term, topn = 30))


# print(tri_test)
df = pd.DataFrame(data = tri_test, index = risk_coterms+practice_coterms)
display(df)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(waterhouse, 0.5382134914398193)","(waterhouse_llp, 0.5271006226539612)","(exercise, 0.5001725554466248)","(collar, 0.48145753145217896)","(upper_strike, 0.47148674726486206)","(nymex_henry_hub, 0.47131994366645813)","(kaufman_kammholz, 0.4659155011177063)","(lme, 0.4496752619743347)","(differential, 0.44930440187454224)","(weighted_averageexercise, 0.4448566138744354)",...,"(floors_collars, 0.4193122386932373)","(conversion, 0.4192864000797272)","(legg_mason_rowe, 0.4180442690849304)","(brent, 0.4179120659828186)","(froze_usf, 0.41334325075149536)","(ceiling, 0.41269612312316895)","(waterhouse_pricewaterhousecoopers, 0.41210162...","(nymex_wti, 0.41164106130599976)","(nbsk_list, 0.4075831472873688)","(gum_rosin, 0.40714436769485474)"
protest,"(protests, 0.7015029191970825)","(protested, 0.6375338435173035)","(usitc, 0.524922788143158)","(icsid, 0.5201950073242188)","(citizen_petition, 0.5169672966003418)","(unsuccessful_bidders, 0.5131667256355286)","(nypsc, 0.5107462406158447)","(respondent, 0.5088033676147461)","(tribunal, 0.5056250691413879)","(reexaminations, 0.4994103014469147)",...,"(windham_defendants, 0.4748821556568146)","(relator, 0.474238783121109)","(opposition, 0.47283828258514404)","(private_litigants, 0.47275403141975403)","(eeoc, 0.4664594531059265)","(rfp, 0.4663366675376892)","(mpuc, 0.46442878246307373)","(appellate, 0.46421018242836)","(puco, 0.46393531560897827)","(sierra_club, 0.4633103013038635)"
injury,"(injuries, 0.7647931575775146)","(personal_injury, 0.6362467408180237)","(damage, 0.6278994679450989)","(personal_injuries, 0.5995531678199768)","(allergic_reactions, 0.5985720157623291)","(dangerous, 0.5957072973251343)","(injured, 0.5909403562545776)","(illness, 0.5901919603347778)","(contamination, 0.5762178897857666)","(incapacitation, 0.5637914538383484)",...,"(harmful, 0.5316370725631714)","(overuse, 0.5310016870498657)","(fatal, 0.5301809906959534)","(sudden_cardiac, 0.5297572612762451)","(injuries_allegedly, 0.529534101486206)","(motor_neurons, 0.5294996500015259)","(endangerment, 0.5286028385162354)","(sickened, 0.526280403137207)","(allergic, 0.5231453776359558)","(harms, 0.5206125378608704)"
delay,"(delays, 0.6547819972038269)","(delayed, 0.6080998182296753)","(halt, 0.5982756614685059)","(delaying, 0.5565288066864014)","(altogether, 0.5468391180038452)","(suspend, 0.5277440547943115)","(disrupt, 0.52545166015625)","(curtail, 0.5226408243179321)","(impede, 0.521216094493866)","(postpone, 0.5049829483032227)",...,"(modify, 0.42217376828193665)","(interrupt_delay, 0.4145295023918152)","(possibly, 0.41408461332321167)","(accelerate, 0.4095895290374756)","(postponing, 0.40103766322135925)","(difficulties, 0.4009297490119934)","(abandon, 0.4001668095588684)","(cause, 0.3970063030719757)","(curtailing, 0.39673110842704773)","(hamper, 0.39389824867248535)"
disruption,"(disruptions, 0.8936829566955566)","(interruptions, 0.7992265224456787)","(interruption, 0.7486926317214966)","(disrupt, 0.658064603805542)","(disrupted, 0.6414289474487305)","(failures, 0.5835986733436584)","(shutdowns, 0.5760209560394287)","(interrupt, 0.5675363540649414)","(problems, 0.5542771220207214)","(unavailability, 0.5494019985198975)",...,"(instability, 0.5163846015930176)","(prolonged, 0.5144786834716797)","(disaster, 0.5138635635375977)","(shutdown, 0.5110733509063721)","(stoppage, 0.507073163986206)","(shortages, 0.5056381225585938)","(compromise, 0.5054807662963867)","(disrupts, 0.5044113993644714)","(disrupting, 0.5033888816833496)","(degradation, 0.496974915266037)"
lawsuit,"(complaint, 0.7108051180839539)","(suit, 0.7042682766914368)","(lawsuits, 0.6651405096054077)","(qui_tam_lawsuit, 0.6592394113540649)","(counterclaim, 0.6372274160385132)","(los_angeles_superior, 0.614996612071991)","(plaintiff, 0.6036272644996643)","(purported, 0.5979205369949341)","(cjc, 0.5933835506439209)","(action, 0.5930635929107666)",...,"(litigation, 0.560692310333252)","(lawsui

In [ ]:

# pair comparison
pairs = [['price', 'value'], ['product','products'],['property','asset'], ['equal', 'equity'], # intuitively similar words
         ['less', 'more'], ['decrease', 'increase'], ['success', 'failure'], ['above', 'below']] # intuitively opposite words
for pair in pairs:
  print(pair, tri_model.wv.similarity(*pair))
  # print(pair, tri_model.wv.relative_cosine_similarity(*pair))
# print(tri_model.wv.most_similar(positive = ['fair_value', 'equity'], negative = ['risk'], topn = 10))

['price', 'value'] 0.3870008
['product', 'products'] 0.6937971
['property', 'asset'] 0.43563345
['equal', 'equity'] 0.0020217928
['less', 'more'] 0.73710114
['decrease', 'increase'] 0.88350606
['success', 'failure'] 0.08719421
['above', 'below'] 0.9018054


In [ ]:
# analogies
def analogy(worda, wordb, wordc):
    result = tri_model.wv.most_similar(negative=[worda], 
                                positive=[wordb, wordc])
    return result

negs = ['less', 'failure', 'below']
analogies = [analogy('decrease', 'increase', neg) for neg in negs]
display(analogies)

[[('greater', 0.7714941501617432),
  ('more', 0.6955163478851318),
  ('reprice_faster', 0.6174420118331909),
  ('children_younger', 0.5983197689056396),
  ('cream_cheese_whipped', 0.5816597938537598),
  ('inconsequential_errors', 0.5793609619140625),
  ('rather', 0.5694777965545654),
  ('sooner', 0.5686042904853821),
  ('faster', 0.5379449129104614),
  ('higher', 0.5356206297874451)],
 [('fail', 0.723248302936554),
  ('inability', 0.6971621513366699),
  ('failing', 0.6286462545394897),
  ('fails', 0.6242179274559021),
  ('failures', 0.566061794757843),
  ('ability', 0.5355926156044006),
  ('failed', 0.5096701979637146),
  ('unable', 0.45846444368362427),
  ('noncompliance', 0.4248608350753784),
  ('attempt', 0.4146870970726013)],
 [('above', 0.8220474720001221),
  ('herein', 0.6390916705131531),
  ('preceding_sentence', 0.6094448566436768),
  ('previous_sentence', 0.5981379747390747),
  ('footnote', 0.5599602460861206),
  ('enchanted', 0.5426034927368164),
  ('therein', 0.5256507396697

## Grid Search for 3-gram model with 3000 samples

In [ ]:
# Set values for various parameters
num_features = [200]    # Word vector dimensionality                      
min_word_count = [10]   # Minimum word count - this limits vocabulary size              
num_workers = 4       # Number of threads to run in parallel
context = [10]          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

def gs(tokenizers, min_count = min_word_count, context_size = context, vector_size = num_features):
  # dataframes
  dfs = []
  
  for min_word_count in min_count:
    for context in context_size:
      for num_features in vector_size:
        print("Training model...")

        # Using default architecture: skip-gram
        tri_model = word2vec.Word2Vec(tokenizers, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling)
        
        # save model
        tri_model_name = "{}features_{}minwords_{}context_trigram".format(num_features, min_word_count, context)
        print("Training model...", tri_model_name)

        # Using default architecture: skip-gram
        tri_model = word2vec.Word2Vec(tri_sentences, workers=num_workers, \
                    size=num_features, min_count = min_word_count, \
                    window = context, sample = downsampling)

        # If you don't plan to train the model any further, calling 
        # init_sims will make the model much more memory-efficient.
        tri_model.init_sims(replace=True)

        # It can be helpful to create a meaningful model name and 
        # save the model for later use. You can load it later using Word2Vec.load()
        
        tri_model.save(tri_model_name)

        
        # top similarities
        # make co-occurrence list
        tri_words = tri_model.wv.index2word
        risk_coterms = check_occ(risk_terms, tri_words)
        practice_coterms = check_occ(practice_terms, tri_words)
        sim_test = []
        for term in risk_coterms+practice_coterms:
          sim_test.append(tri_model.wv.most_similar(term, topn = 30))
        
        # df_sim = pd.DataFrame(data = sim_test, index = risk_coterms+practice_coterms, columns = [1,2,3,4,5])
        df_sim = pd.DataFrame(data = sim_test, index = risk_coterms+practice_coterms)
        dfs.append(df_sim)

        # # pair test
        # pair_test = []
        # for pair in pairs:
        #     pair_test.append(tri_model.wv.similarity(*pair))
            
        # df_pair = pd.DataFrame(data = pair_test, index = pairs)
        # dfs.append(df_pair)

        # to file
        df_sim.to_csv('{}.csv'.format(tri_model_name))
        
  return dfs

### search for num_features

In [ ]:
nf_gs = gs(tri_sentences, vector_size = [50, 100, 200, 300])


2022-03-31 15:58:23,963 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2022-03-31 15:58:23,972 : INFO : collecting all words and their counts
2022-03-31 15:58:23,976 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-31 15:58:24,140 : INFO : PROGRESS: at sentence #10000, processed 273328 words, keeping 9843 word types


Training model...


2022-03-31 15:58:24,185 : INFO : collected 10689 word types from a corpus of 326173 raw words and 11949 sentences
2022-03-31 15:58:24,193 : INFO : Loading a fresh vocabulary
2022-03-31 15:58:24,225 : INFO : effective_min_count=10 retains 3567 unique words (33% of original 10689, drops 7122)
2022-03-31 15:58:24,241 : INFO : effective_min_count=10 leaves 300696 word corpus (92% of original 326173, drops 25477)
2022-03-31 15:58:24,267 : INFO : deleting the raw counts dictionary of 10689 items
2022-03-31 15:58:24,276 : INFO : sample=0.001 downsamples 37 most-common words
2022-03-31 15:58:24,284 : INFO : downsampling leaves estimated 209786 word corpus (69.8% of prior 300696)
2022-03-31 15:58:24,310 : INFO : estimated required memory for 3567 words and 50 dimensions: 3210300 bytes
2022-03-31 15:58:24,318 : INFO : resetting layer weights
2022-03-31 15:58:27,263 : INFO : training model with 4 workers on 3567 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2022-03

ValueError: ignored

In [ ]:
# from google.colab import data_table
# from vega_datasets import data

# data_table.enable_dataframe_formatter()

for df in nf_gs:
  display(df)

,1,2,3,4,5
delay,"(potential, 0.9902563095092773)","(cause, 0.9892822504043579)","(prevent, 0.9855485558509827)","(financing, 0.983407735824585)","(substantial, 0.9826462864875793)"
disruption,"(interruption, 0.9880797863006592)","(reputation, 0.9880272150039673)","(could_harm, 0.9867870807647705)","(prospects, 0.983432412147522)","(be_harmed, 0.9831605553627014)"
lawsuit,"(submission, 0.9883208870887756)","(rule, 0.9875807166099548)","(update, 0.9856951236724854)","(written, 0.9839335680007935)","(administered, 0.9834583401679993)"
litigation,"(factors, 0.9881578683853149)","(substantial, 0.9865521192550659)","(proprietary_information, 0.9826984405517578)","(substantial_costs, 0.982205331325531)","(delay, 0.980819582939148)"
court,"(securing, 0.9936972260475159)","(give, 0.9936701059341431)","(disclose, 0.9931272864341736)","(way, 0.9926424622535706)","(status, 0.9923014640808105)"
enforcement,"(challenging, 0.9982675909996033)","(error, 0.9971636533737183)","(inherent, 0.9970351457595825)","(member_states, 0.996740460395813)","(involved, 0.9965382814407349)"
transparency,"(concerns, 0.9916974902153015)","(federal_state, 0.9907255172729492)","(financial_institutions, 0.9883411526679993)","(extensive, 0.9883086085319519)","(authorities, 0.9880056381225586)"
program,"(secretary, 0.9947683215141296)","(north, 0.994176983833313)","(focused_on, 0.9941734075546265)","(financial_instruments, 0.9937654733657837)","(tennessee, 0.9937241077423096)"
training,"(efficiency, 0.9970887899398804)","(concepts, 0.996817409992218)","(processing, 0.9966902136802673)","(production, 0.9958635568618774)","(biopharmaceutical, 0.9958111047744751)"
recruit,"(grow, 0.9919538497924805)","(leverage, 0.9894872903823853)","(opportunities, 0.988794207572937)","(covenants, 0.9851241111755371)","(economic, 0.9849851727485657)"


,1,2,3,4,5
delay,"(potential, 0.9935972690582275)","(disruptions, 0.993572473526001)","(cause, 0.9926067590713501)","(materially, 0.9901942014694214)","(prevent, 0.9891904592514038)"
disruption,"(prospects, 0.9927528500556946)","(liquidity, 0.9923210740089417)","(business_strategy, 0.9920666217803955)","(interruption, 0.9910269379615784)","(could_harm, 0.9908700585365295)"
lawsuit,"(american, 0.9937363266944885)","(diagnosis, 0.993346095085144)","(decision, 0.9927793741226196)","(utilizes, 0.9922217726707458)","(medicine, 0.992078423500061)"
litigation,"(financing, 0.9891688823699951)","(ways, 0.9863836765289307)","(materially, 0.9849660992622375)","(delay, 0.9838640689849854)","(potential, 0.9827524423599243)"
court,"(claim, 0.9962016344070435)","(challenge, 0.9961870908737183)","(way, 0.9958405494689941)","(properly, 0.9956757426261902)","(widely, 0.9955530166625977)"
enforcement,"(error, 0.9978193640708923)","(varying, 0.9974512457847595)","(involving, 0.9972190260887146)","(monetary, 0.997215211391449)","(highly, 0.9970373511314392)"
transparency,"(authorities, 0.9963482022285461)","(financial_institutions, 0.9960367679595947)","(notification, 0.9946210384368896)","(gdpr, 0.9944876432418823)","(fraud, 0.9941079020500183)"
program,"(japan, 0.9969061613082886)","(reconciliation, 0.9966722130775452)","(flagship, 0.9964640736579895)","(solid_tumors, 0.9961525201797485)","(discovery, 0.9960907101631165)"
training,"(retail, 0.9970411062240601)","(truck, 0.9954499006271362)","(processing, 0.9945367574691772)","(buildings, 0.9942061901092529)","(areas, 0.9941859245300293)"
recruit,"(success, 0.9939679503440857)","(commercialization_efforts, 0.9935429096221924)","(leverage, 0.993092954158783)","(expansion, 0.9921010136604309)","(debts, 0.9920153617858887)"


,1,2,3,4,5
delay,"(affect, 0.9941512942314148)","(be_harmed, 0.9938084483146667)","(potential, 0.9937044382095337)","(significantly, 0.9895234107971191)","(disruptions, 0.9886951446533203)"
disruption,"(effect_on, 0.9918332695960999)","(a_material_adverse, 0.9882235527038574)","(adversely_affect, 0.9868451356887817)","(operating_results, 0.9850082397460938)","(materially_adversely_affected, 0.984359681606..."
lawsuit,"(considers, 0.9977002739906311)","(recommendation, 0.996719241142273)","(dependent, 0.9963011741638184)","(competent_authorities, 0.9958605766296387)","(european_commission, 0.9958046674728394)"
litigation,"(potential, 0.9929271936416626)","(financing, 0.9914700984954834)","(delays, 0.9913923740386963)","(compete, 0.9909148216247559)","(diminish, 0.9901179075241089)"
court,"(instrument, 0.9974147081375122)","(pivotal, 0.9968546628952026)","(properly, 0.9966435432434082)","(brought, 0.99658203125)","(matter, 0.996523916721344)"
enforcement,"(regularly, 0.9980932474136353)","(varying, 0.9978224635124207)","(understand, 0.9977090954780579)","(nature, 0.9976851940155029)","(disclosures, 0.99765545129776)"
transparency,"(prohibit, 0.9977550506591797)","(health_information, 0.9970771670341492)","(insurers, 0.996859073638916)","(financial_institutions, 0.9957610368728638)","(work, 0.9948102235794067)"
program,"(bonuses, 0.9974751472473145)","(office, 0.9953111410140991)","(huntsville, 0.9946618676185608)","(savings, 0.9942078590393066)","(colitis, 0.9937896728515625)"
training,"(encourage, 0.9968309998512268)","(consultants, 0.9967226982116699)","(build, 0.9965585470199585)","(fuel, 0.9964276552200317)","(other_reits, 0.9960346817970276)"
recruit,"(success, 0.997345507144928)","(diversify, 0.9960971474647522)","(focus_on, 0.9954286217689514)","(prospective, 0.9942665696144104)","(pursue, 0.9941431283950806)"


,1,2,3,4,5
delay,"(negatively_affect, 0.9967248439788818)","(profitability, 0.9955399036407471)","(disruptions, 0.9953175783157349)","(be_harmed, 0.9950025081634521)","(cause, 0.9949460029602051)"
disruption,"(could_adversely_affect, 0.9953094124794006)","(could_harm, 0.9940609931945801)","(prospects, 0.9935891628265381)","(could_materially, 0.992306649684906)","(operating_results, 0.9920663237571716)"
lawsuit,"(evaluation, 0.9966491460800171)","(effective_january, 0.9960410594940186)","(as_defined, 0.9958218336105347)","(servicenow, 0.995728611946106)","(administered, 0.9956833124160767)"
litigation,"(reasons, 0.9953778982162476)","(be_adversely_affected, 0.9946866035461426)","(taking, 0.9935795068740845)","(ways, 0.9924540519714355)","(compete, 0.9920039176940918)"
court,"(though, 0.9979056715965271)","(country, 0.9977213144302368)","(instead, 0.9976352453231812)","(pending, 0.9975733160972595)","(executing, 0.9973881840705872)"
enforcement,"(personal, 0.999135434627533)","(interpretation, 0.998881459236145)","(important, 0.9988307952880859)","(affecting, 0.9983820915222168)","(involving, 0.9981439113616943)"
transparency,"(hipaa, 0.9975993633270264)","(prohibit, 0.9964355826377869)","(notification, 0.9956639409065247)","(concerns, 0.9954996109008789)","(authorities, 0.995207667350769)"
program,"(iii, 0.9978145956993103)","(jpp_llc, 0.9977278709411621)","(entry, 0.9973236322402954)","(monthly, 0.9966159462928772)","(jpp_ii_llc, 0.9961287975311279)"
training,"(records, 0.9955017566680908)","(supply, 0.995309591293335)","(storage, 0.9950311183929443)","(healthcare_programs, 0.9949556589126587)","(items, 0.994823694229126)"
recruit,"(about, 0.995166540145874)","(strategies, 0.9947075247764587)","(managing, 0.9946969151496887)","(our_hotel_managers, 0.9945592284202576)","(developments, 0.9943525791168213)"


### grid search for context window size


In [ ]:
cs_gs = gs(tri_sentences, context_size = [5, 10, 20])

2022-03-30 14:13:52,566 : INFO : collecting all words and their counts
2022-03-30 14:13:52,569 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-30 14:13:52,627 : INFO : PROGRESS: at sentence #10000, processed 273328 words, keeping 9843 word types
2022-03-30 14:13:52,640 : INFO : collected 10689 word types from a corpus of 326173 raw words and 11949 sentences
2022-03-30 14:13:52,642 : INFO : Loading a fresh vocabulary
2022-03-30 14:13:52,655 : INFO : effective_min_count=10 retains 3567 unique words (33% of original 10689, drops 7122)
2022-03-30 14:13:52,657 : INFO : effective_min_count=10 leaves 300696 word corpus (92% of original 326173, drops 25477)
2022-03-30 14:13:52,672 : INFO : deleting the raw counts dictionary of 10689 items
2022-03-30 14:13:52,674 : INFO : sample=0.001 downsamples 37 most-common words
2022-03-30 14:13:52,675 : INFO : downsampling leaves estimated 209786 word corpus (69.8% of prior 300696)
2022-03-30 14:13:52,686 : INFO : estima

Training model...


2022-03-30 14:13:53,315 : INFO : training model with 4 workers on 3567 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-03-30 14:13:53,592 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:53,600 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:53,606 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:53,610 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-30 14:13:53,611 : INFO : EPOCH - 1 : training on 326173 raw words (209805 effective words) took 0.3s, 729267 effective words/s
2022-03-30 14:13:53,886 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:53,890 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:53,905 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:53,913 : INFO : worker thread finished; awaiting finish of 0 m

Training model...


2022-03-30 14:13:55,619 : INFO : training model with 4 workers on 3567 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2022-03-30 14:13:55,920 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:55,943 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:55,950 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:55,959 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-30 14:13:55,960 : INFO : EPOCH - 1 : training on 326173 raw words (210017 effective words) took 0.3s, 630601 effective words/s
2022-03-30 14:13:56,268 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:56,278 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:56,279 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:56,285 : INFO : worker thread finished; awaiting finish of 0 

Training model...


2022-03-30 14:13:58,120 : INFO : training model with 4 workers on 3567 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=20
2022-03-30 14:13:58,467 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:58,483 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:58,493 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:58,498 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-30 14:13:58,499 : INFO : EPOCH - 1 : training on 326173 raw words (209720 effective words) took 0.4s, 564785 effective words/s
2022-03-30 14:13:58,847 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-30 14:13:58,849 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-30 14:13:58,865 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-30 14:13:58,871 : INFO : worker thread finished; awaiting finish of 0 

In [ ]:
for df in cs_gs:
  display(df)

NameError: ignored

### Grid Search over min_count, vector size, window size

In [ ]:
# running time for 2 * 3 * 4 -- 3m37s
all_gs = gs(tri_sentences, min_count = [2, 5], context_size = [5, 10, 15], vector_size = [50, 100, 150, 200])



2022-04-04 13:31:34,234 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2022-04-04 13:31:34,242 : INFO : collecting all words and their counts
2022-04-04 13:31:34,246 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-04 13:31:34,417 : INFO : PROGRESS: at sentence #10000, processed 273729 words, keeping 9881 word types


Training model...


2022-04-04 13:31:34,461 : INFO : collected 10727 word types from a corpus of 326578 raw words and 11949 sentences
2022-04-04 13:31:34,467 : INFO : Loading a fresh vocabulary
2022-04-04 13:31:34,525 : INFO : effective_min_count=2 retains 8662 unique words (80% of original 10727, drops 2065)
2022-04-04 13:31:34,531 : INFO : effective_min_count=2 leaves 324513 word corpus (99% of original 326578, drops 2065)
2022-04-04 13:31:34,592 : INFO : deleting the raw counts dictionary of 10727 items
2022-04-04 13:31:34,599 : INFO : sample=0.001 downsamples 34 most-common words
2022-04-04 13:31:34,605 : INFO : downsampling leaves estimated 235760 word corpus (72.7% of prior 324513)
2022-04-04 13:31:34,666 : INFO : estimated required memory for 8662 words and 50 dimensions: 7795800 bytes
2022-04-04 13:31:34,674 : INFO : resetting layer weights
2022-04-04 13:31:38,488 : INFO : training model with 4 workers on 8662 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-04-04 

Training model...


2022-04-04 13:31:41,670 : INFO : deleting the raw counts dictionary of 10727 items
2022-04-04 13:31:41,678 : INFO : sample=0.001 downsamples 34 most-common words
2022-04-04 13:31:41,684 : INFO : downsampling leaves estimated 235760 word corpus (72.7% of prior 324513)
2022-04-04 13:31:41,729 : INFO : estimated required memory for 8662 words and 50 dimensions: 7795800 bytes
2022-04-04 13:31:41,736 : INFO : resetting layer weights
2022-04-04 13:31:45,126 : INFO : training model with 4 workers on 8662 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-04-04 13:31:45,538 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:31:45,541 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:31:45,549 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:31:45,553 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:31:45,555 : INFO : EPOCH - 1 : training on

...

2022-04-04 13:34:58,683 : INFO : effective_min_count=5 retains 5883 unique words (54% of original 10727, drops 4844)
2022-04-04 13:34:58,692 : INFO : effective_min_count=5 leaves 316668 word corpus (96% of original 326578, drops 9910)
2022-04-04 13:34:58,731 : INFO : deleting the raw counts dictionary of 10727 items
2022-04-04 13:34:58,733 : INFO : sample=0.001 downsamples 36 most-common words
2022-04-04 13:34:58,739 : INFO : downsampling leaves estimated 227222 word corpus (71.8% of prior 316668)
2022-04-04 13:34:58,775 : INFO : estimated required memory for 5883 words and 150 dimensions: 10001100 bytes
2022-04-04 13:34:58,777 : INFO : resetting layer weights
2022-04-04 13:35:00,492 : INFO : training model with 4 workers on 5883 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:35:01,092 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:35:01,100 : INFO : worker thread finished; awaiting finish of 2 more threads

Training model...


2022-04-04 13:35:05,049 : INFO : training model with 4 workers on 5883 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:35:05,547 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:35:05,554 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:35:05,560 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:35:05,564 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:35:05,566 : INFO : EPOCH - 1 : training on 326578 raw words (226955 effective words) took 0.5s, 449421 effective words/s
2022-04-04 13:35:06,035 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:35:06,044 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:35:06,059 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:35:06,079 : INFO : worker thread finished; awaiting finish of 0 

Training model...


2022-04-04 13:35:07,931 : INFO : effective_min_count=5 retains 5883 unique words (54% of original 10727, drops 4844)
2022-04-04 13:35:07,934 : INFO : effective_min_count=5 leaves 316668 word corpus (96% of original 326578, drops 9910)
2022-04-04 13:35:07,961 : INFO : deleting the raw counts dictionary of 10727 items
2022-04-04 13:35:07,963 : INFO : sample=0.001 downsamples 36 most-common words
2022-04-04 13:35:07,966 : INFO : downsampling leaves estimated 227222 word corpus (71.8% of prior 316668)
2022-04-04 13:35:07,992 : INFO : estimated required memory for 5883 words and 200 dimensions: 12354300 bytes
2022-04-04 13:35:07,997 : INFO : resetting layer weights
2022-04-04 13:35:09,109 : INFO : training model with 4 workers on 5883 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:35:09,595 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:35:09,609 : INFO : worker thread finished; awaiting finish of 2 more threads

In [ ]:
for df in gs:
  display(df)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(installments, 0.9799250364303589)","(on_early_extinguishment, 0.9765149354934692)","(interest_rate_cap, 0.9720622301101685)","(total, 0.9712532162666321)","(measurement_period, 0.9704278707504272)","(criteria_established, 0.9695432186126709)","(total_revenues, 0.967095136642456)","(billion, 0.9656417965888977)","(ending, 0.9633399844169617)","(year, 0.9631911516189575)",...,"(rou_assets, 0.953443706035614)","(requested_mortgages, 0.95325767993927)","(stabilization, 0.952283501625061)","(approximately_million, 0.9520754814147949)","(was, 0.9520525932312012)","(vesting_period, 0.9518686532974243)","(initial_funding, 0.951137125492096)","(milestone_payment, 0.950969934463501)","(sublease, 0.9485681056976318)","(bla_submission, 0.9475923776626587)"
injury,"(civil_penalties, 0.9960111379623413)","(threats, 0.995396077632904)","(institutions, 0.9951409697532654)","(national, 0.9951003193855286)","(cells, 0.9949293732643127)","(severity, 0.9949206113815308)","(imposes, 0.9946600794792175)","(insurers, 0.9943538904190063)","(suggest_that, 0.9943351149559021)","(recent, 0.9941715598106384)",...,"(helps, 0.9934468865394592)","(il, 0.9934216141700745)","(quick_service, 0.9933630228042603)","(false_claims_act, 0.9933308362960815)","(normal, 0.9932502508163452)","(natural_disasters, 0.9931715130805969)","(personal, 0.9931151866912842)","(user, 0.9930171370506287)","(fraudulent, 0.9930018186569214)","(commercial_real_estate, 0.9929991364479065)"
delay,"(compromise, 0.9920529723167419)","(commercialize, 0.9915096759796143)","(manage, 0.9913215637207031)","(take, 0.9911646842956543)","(cause, 0.9911206960678101)","(patents, 0.990786612033844)","(additional_indebtedness, 0.9906625747680664)","(financing, 0.9906151294708252)","(execute, 0.9903846979141235)","(factors, 0.9898444414138794)",...,"(prevent, 0.9880238771438599)","(pursue, 0.9879900813102722)","(weaken, 0.9877877235412598)","(insufficient, 0.9876896142959595)","(replace, 0.987395167350769)","(significantly, 0.9873846769332886)","(lead, 0.9873613715171814)","(funds, 0.9870087504386902)","(willingness, 0.9869970679283142)","(actual_results, 0.9869903922080994)"
disruption,"(liquidity, 0.9861958622932434)","(growth, 0.9861112236976624)","(capital, 0.9830780625343323)","(debt, 0.9829298257827759)","(turn, 0.9826574325561523)","(grow, 0.9813475012779236)","(potential, 0.98050457239151)","(competitive_position, 0.9786577224731445)","(repay, 0.9781486392021179)","(pay_distributions, 0.9776232242584229)",...,"(will_depend, 0.9736723899841309)","(differ_from, 0.9732236862182617)","(effectively_manage, 0.9730348587036133)","(decline, 0.9724023342132568)","(commercialization_efforts, 0.9719412922859192)","(indebtedness, 0.9717422723770142)","(damage, 0.9716916680335999)","(harm, 0.9715460538864136)","(losses, 0.9711549282073975)","(deduct, 0.971086859703064)"
lawsuit,"(stockholder, 0.9919877648353577)","(institution, 0.9913743138313293)","(nonfinancial, 0.9897048473358154)","(under_section, 0.9892845749855042)","(until, 0.9891897439956665)","(bankruptcy_court_entered, 0.9887323379516602)","(combined_performance_obligation, 0.9886177182...","(satisfies, 0.9886125922203064)","(volume_variance, 0.9881255030632019)","(performance_obligation, 0.9880960583686829)",...,"(requisite_service_period, 0.9855163097381592)","(holder, 0.985460102558136)","(beneficially_owns, 0.9852485060691833)","(dr_ausiello, 0.9848757982254028)","(utilizes, 0.9848722815513611)","(cumulative_effect, 0.9848604202270508)","(final, 0.9846627712249756)","(formula, 0.9846465587615967)","(following, 0.9845724105834961)","(alll, 0.9845701456069946)"
litigation,"(claims_against, 0.9956522583961487)","(factors, 0.9951529502868652)","(intellectual_property_rights, 0.9930182099342...","(affected_by, 0.9925839304924011)","(engage, 0.9921492338180542)","(prohibit, 0.9916978478431702)","(deficiencies, 0.9914772510528564)","(identify, 0.9911693930625916)","(responsible_for, 0.

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(notional_amount, 0.9918954372406006)","(third, 0.9915982484817505)","(ten, 0.9909794926643372)","(transaction, 0.9905147552490234)","(shareholder_return, 0.9898933172225952)","(balance, 0.9887838363647461)","(more_than, 0.9887017011642456)","(previous, 0.9884772896766663)","(net_worth, 0.9882904291152954)","(weighted_average, 0.9874662756919861)",...,"(revalued, 0.9862011671066284)","(new_york_stock, 0.9858814477920532)","(three, 0.9857698082923889)","(maryland_reit, 0.9856730699539185)","(second, 0.9856446981430054)","(total_average, 0.9855543971061707)","(trial, 0.9855469465255737)","(milestone, 0.9854938387870789)","(difference_between, 0.9854785799980164)","(goodwill, 0.9854551553726196)"
injury,"(davidson, 0.9982292652130127)","(clinical_benefit, 0.9980877637863159)","(pass, 0.9980363845825195)","(killing, 0.9979963898658752)","(tumor, 0.9979656338691711)","(active, 0.9979479908943176)","(antigens, 0.9978808164596558)",(travelcentersofamericamember_svc_travelcenter...,"(bcg, 0.9977602958679199)","(county, 0.9976839423179626)",...,"(o, 0.997106671333313)","(hydraulic, 0.9971015453338623)","(diagnostic, 0.997098982334137)","(hoover, 0.9970954656600952)","(counties, 0.9970858097076416)","(stops, 0.9970812201499939)","(sf, 0.9970616102218628)","(every, 0.9970169067382812)","(colitis, 0.9969786405563354)","(learning, 0.9969704151153564)"
delay,"(cause, 0.9943705797195435)","(downgrade, 0.9943357110023499)","(compromise, 0.9942549467086792)","(infringement, 0.9932052493095398)","(purchasers, 0.9927321672439575)","(patents, 0.9922292232513428)","(financing, 0.9919707775115967)","(manage, 0.9912225604057312)","(third_party_manufacturer, 0.9907391667366028)","(commercialize, 0.9906806349754333)",...,"(relate, 0.9887335896492004)","(variations, 0.9886966943740845)","(profitability, 0.9886264801025391)","(additional_indebtedness, 0.9884116649627686)","(proprietary_information, 0.9883904457092285)","(retain, 0.9883173704147339)","(turn, 0.9881086349487305)","(more_difficult, 0.9879603385925293)","(negatively_impact, 0.9878567457199097)","(deployed, 0.9877548217773438)"
disruption,"(will_depend, 0.989075779914856)","(history, 0.9827553033828735)","(incurring_additional, 0.9825590252876282)","(misstatements, 0.9816334247589111)","(confidence, 0.9809417724609375)","(liquidity, 0.9797303080558777)","(startups, 0.9782747030258179)","(imposition, 0.9780394434928894)","(allocable, 0.9779011011123657)","(omissions, 0.9757421016693115)",...,"(conflicts, 0.9726429581642151)","(transforming, 0.9724951386451721)","(operation, 0.9723085165023804)","(key_personnel, 0.9721723198890686)","(substantially_all, 0.9721085429191589)","(vacancies, 0.972050666809082)","(success, 0.9719358682632446)","(prospects, 0.9717854261398315)","(disposition, 0.9716353416442871)","(debt, 0.9716331362724304)"
lawsuit,"(receives, 0.9938381910324097)","(dysbiosis, 0.9932729601860046)","(condition, 0.9930928349494934)","(derivative, 0.9925346970558167)","(determination, 0.9925306439399719)","(then, 0.9924119710922241)","(reportable, 0.9920074939727783)","(shareholder, 0.9918619394302368)","(instrument, 0.9918519258499146)","(subsidiary, 0.9915812611579895)",...,"(greater_than, 0.9900645017623901)","(bla, 0.9899532198905945)","(consideration, 0.9896246194839478)","(secured, 0.9895665645599365)","(partial_motion, 0.9895632266998291)","(promise, 0.9895180463790894)","(noted, 0.9893643260002136)","(adjustment, 0.9892983436584473)","(performance_obligation, 0.98918616771698)","(secured_by, 0.9890125393867493)"
litigation,"(estimates, 0.995025098323822)","(moreover, 0.9950172305107117)","(provisions, 0.9949793815612793)","(complete, 0.994949996471405)","(compliance_with, 0.9948470592498779)","(offset, 0.9946534633636475)","(demands, 0.994493842124939)","(our_licensors, 0.9942169785499573)","(patent_rights, 0.9941956996917725)","(premises, 0.9941833019256592)",...,"(without, 0.9932098984718323)","(action, 0.99318403

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(gain, 0.9886369109153748)","(loss, 0.9873157739639282)","(public_offering, 0.9863148927688599)","(total_revenues, 0.9842585921287537)","(absence, 0.9839365482330322)","(percentage, 0.9827225804328918)","(approximately_million, 0.9816312789916992)","(exercise_price, 0.9807772040367126)","(total, 0.9806336164474487)","(approximately, 0.9803954362869263)",...,"(owned, 0.9763710498809814)","(settled, 0.9762684106826782)","(issuance, 0.9754595756530762)","(net, 0.9746367931365967)","(completed, 0.9740563631057739)","(income, 0.9735680222511292)","(principal_amount, 0.9733269214630127)","(expense, 0.9730616211891174)","(decrease, 0.9728060960769653)","(phase_b_clinical, 0.9727324843406677)"
injury,"(human, 0.9988324046134949)","(pharmaceutical_companies, 0.9985082745552063)","(scale, 0.9984314441680908)","(georgia, 0.9981275796890259)","(h, 0.9980911612510681)","(healthcare, 0.9980146288871765)","(identifying, 0.9978954792022705)","(brands, 0.997883141040802)","(incorporation, 0.9978770613670349)","(street_suite, 0.9978432059288025)",...,"(formulation, 0.9976040124893188)","(spore, 0.9976003766059875)","(quality, 0.9975650310516357)","(unauthorized, 0.9975239038467407)","(flagship_fund_vi, 0.9975078105926514)","(training, 0.9975060224533081)","(brazil, 0.9974748492240906)","(affairs, 0.9974689483642578)","(holding, 0.9974076151847839)","(denial, 0.9973989725112915)"
delay,"(supplement, 0.9963146448135376)","(collaborators, 0.9961725473403931)","(licensees, 0.9960445761680603)","(profitability, 0.9951602220535278)","(additional_indebtedness, 0.994602620601654)","(exist, 0.9945504069328308)","(patents, 0.9940752983093262)","(compromise, 0.9937474727630615)","(insufficient, 0.9934834241867065)","(experience, 0.9933627247810364)",...,"(potential, 0.9908860921859741)","(cease, 0.9907239079475403)","(commercialize, 0.9906757473945618)","(relate, 0.9905989170074463)","(preclude, 0.9903932809829712)","(their_businesses, 0.9901150465011597)","(future_legislation, 0.9900839924812317)","(new, 0.9899542927742004)","(further, 0.9899283647537231)","(execute, 0.9898800849914551)"
disruption,"(stock_price, 0.9893490672111511)","(decline, 0.9886613488197327)","(cash_flows, 0.9841703772544861)","(relevance, 0.9823799133300781)","(could_adversely_affect, 0.9802097082138062)","(interruption, 0.9792288541793823)","(indicative, 0.9738360643386841)","(liquidity, 0.9735669493675232)","(diminish, 0.97336345911026)","(dependent_on, 0.9730727672576904)",...,"(implied, 0.9675143957138062)","(allocable, 0.9672541618347168)","(cost, 0.9671548008918762)","(be_harmed, 0.9656866192817688)","(discussion, 0.9653940796852112)","(dispose, 0.9652867913246155)","(top_line, 0.9651823043823242)","(focus, 0.9650968909263611)","(losses, 0.9647147059440613)","(adversely_affect, 0.9644759893417358)"
lawsuit,"(description, 0.9963802695274353)","(non_accelerated_filer, 0.9949653148651123)","(acquirer, 0.9947543144226074)","(large_accelerated_filer, 0.9943274855613708)","(meets, 0.994111180305481)","(marketing_application, 0.9940385222434998)","(receives_marketing, 0.9939410090446472)","(submission, 0.9936845302581787)","(donor, 0.9935998916625977)","(nonfinancial, 0.9935197234153748)",...,"(until, 0.9922038912773132)","(several_years, 0.9921938180923462)","(constitutes, 0.99213045835495)","(stockholder, 0.9921096563339233)","(is_probable, 0.9919797778129578)","(designated, 0.9918599128723145)","(g, 0.991831362247467)","(systemically_administered, 0.9918099045753479)","(healthy, 0.9916766881942749)","(type, 0.9916574358940125)"
litigation,"(moreover, 0.9970495700836182)","(asserting, 0.9956231713294983)","(against, 0.9955111742019653)","(future_legislation, 0.9949049353599548)","(contain, 0.9948731064796448)","(substantial, 0.9947453141212463)","(incurring, 0.9947071671485901)","(forward_looking_statements, 0.9944615960121155)","(similarly, 0.9941791892051697)","(discourage, 0.9937418699264526)",...,"(agencies, 0.992

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(were_billion, 0.994532585144043)","(previous, 0.9935715198516846)","(stabilization, 0.9929553270339966)","(gross_proceeds, 0.9928309917449951)","(those_fiscal, 0.9927694201469421)","(borrowings_under, 0.9925505518913269)","(jobs_act, 0.9918246269226074)","(total_revenues, 0.9914568066596985)","(trading_days, 0.9911829233169556)","(three, 0.9909557104110718)",...,"(placebo, 0.990021824836731)","(purchase_price, 0.9899520874023438)","(phase, 0.9895544648170471)","(unsecured_term, 0.9895360469818115)","(slightly, 0.9894624948501587)","(sublease, 0.9894276857376099)","(fold, 0.9892972707748413)","(fixtures, 0.9891594648361206)","(second, 0.9891190528869629)","(all_material_respects, 0.9889158010482788)"
injury,"(concepts, 0.9986941814422607)","(new_drug, 0.9986846446990967)","(consumers, 0.9984416961669922)","(store, 0.9984054565429688)","(building, 0.998358428478241)","(states, 0.9982919096946716)","(networks, 0.9981980323791504)","(human, 0.9981769919395447)","(mezzanine_finance_llc, 0.9981177449226379)","(seritage_kmt_mezzanine_finance, 0.99807977676...",...,"(dental, 0.9978415966033936)","(msa, 0.9978347420692444)","(florida, 0.9978266954421997)","(multifamily, 0.9978175759315491)","(rental, 0.9978100657463074)","(efficiency, 0.9977537989616394)","(ca, 0.9977512359619141)","(regional, 0.9977375268936157)","(immune, 0.9977280497550964)","(solid, 0.997694730758667)"
delay,"(patents, 0.9955040216445923)","(compromise, 0.994976282119751)","(lead, 0.9947736263275146)","(additional_indebtedness, 0.9945679306983948)","(commercialize, 0.9944798350334167)","(delays, 0.9944553375244141)","(intellectual_property_rights, 0.9944331049919...","(successfully, 0.9943379759788513)","(volatile, 0.9941748380661011)","(factors, 0.9940140247344971)",...,"(contamination, 0.9929020404815674)","(preclude, 0.9928209781646729)","(litigation, 0.9927823543548584)","(manage, 0.9926891326904297)","(material_weaknesses, 0.9924783706665039)","(renew_their, 0.9924412369728088)","(complementary, 0.9923893213272095)","(retain, 0.9923455715179443)","(future_legislation, 0.9922575950622559)","(collaborators, 0.9922381639480591)"
disruption,"(decline, 0.9906250238418579)","(commercial_prospects, 0.9902070760726929)","(stock_price, 0.9860706329345703)","(debt, 0.9823073148727417)","(focus, 0.9821162223815918)","(liquidity, 0.9815645217895508)","(retroactively, 0.9815232157707214)","(growth_prospects, 0.9802456498146057)","(cash_flows, 0.9799097776412964)","(mortgaged, 0.9797032475471497)",...,"(behalf, 0.9769038558006287)","(violation, 0.9765504002571106)","(suspend, 0.9758734703063965)","(conduct, 0.9756745100021362)","(direct, 0.9749571084976196)","(winding_up, 0.9749338626861572)","(trading, 0.9747956991195679)","(an_adverse, 0.97434401512146)","(competitive_position, 0.9742428064346313)","(clinical_trials, 0.9742190837860107)"
lawsuit,"(entity, 0.9963493347167969)","(gap, 0.995093822479248)","(additionally, 0.9948899745941162)","(designation, 0.9947476983070374)","(once, 0.9946296811103821)","(constitute, 0.9939954280853271)","(derivative, 0.9938119649887085)","(payload, 0.9935485124588013)","(non_erisa_plan, 0.9934389591217041)","(unless, 0.9933221340179443)",...,"(pivotal, 0.9917895197868347)","(modified, 0.991750180721283)","(indicated, 0.9917410016059875)","(designate, 0.9917032122612)","(tests, 0.9916179180145264)","(non_u, 0.9915432929992676)","(ultimate, 0.9915209412574768)","(audit_committee, 0.9914684295654297)","(exists, 0.9912369251251221)","(prevents, 0.991166889667511)"
litigation,"(substantial, 0.9980882406234741)","(patent_protection, 0.9979240894317627)","(factors, 0.9974974989891052)","(further, 0.9972927570343018)","(future_legislation, 0.9972120523452759)","(claims, 0.9971649646759033)","(trade_secrets, 0.996553897857666)","(issued_patents, 0.9964032173156738)","(instances, 0.9963115453720093)","(mitigate, 0.9962655901908875)",...,"(whom, 0.9955397844314575)","(take, 0.9955128431

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(measured, 0.977147102355957)","(commitment, 0.9717880487442017)","(parcels, 0.9717649221420288)","(score, 0.9709790945053101)","(return_per_share, 0.9705647230148315)","(endoscopic, 0.9697033762931824)","(initial_funding, 0.9690564870834351)","(trial, 0.9679635167121887)","(verses, 0.9666520357131958)","(january, 0.9640724658966064)",...,"(inclusive, 0.959640383720398)","(commencing, 0.9593790173530579)","(loss_per_share, 0.958916187286377)","(wholly_owned, 0.9583539366722107)","(deemed_probable, 0.9577652812004089)","(second, 0.9577539563179016)","(disease_recurrence, 0.9576612114906311)","(years, 0.9575176239013672)","(first, 0.9573601484298706)","(amortized, 0.9573327302932739)"
injury,"(human, 0.9961662888526917)","(wall_street, 0.9960677623748779)","(instruments, 0.9960513710975647)","(h, 0.9960219860076904)","(led, 0.9959139227867126)","(monitoring, 0.9957712888717651)","(biotechnology, 0.9956639409065247)","(kmart, 0.9956026673316956)","(improvement, 0.995517909526825)","(rebates, 0.9954918026924133)",...,"(peer, 0.9949571490287781)","(removal, 0.9949086904525757)","(address, 0.9948077201843262)","(live, 0.9946847558021545)","(united_kingdom, 0.9946684837341309)","(manual, 0.9946471452713013)","(functional, 0.9945551156997681)","(holding, 0.9944888949394226)","(die, 0.9944829344749451)","(throughout, 0.9944652915000916)"
delay,"(trading_volume, 0.9919119477272034)","(affect, 0.988976240158081)","(materially, 0.9884099960327148)","(invalidate, 0.9881331324577332)","(some, 0.9876344203948975)","(limit, 0.9875500798225403)","(significantly, 0.9866208434104919)","(purchasers, 0.9855470061302185)","(renew_their, 0.9849435687065125)","(potential, 0.9847991466522217)",...,"(other_factors, 0.9801828861236572)","(proprietary_information, 0.9800457954406738)","(reasons, 0.9796023368835449)","(delays, 0.9795324802398682)","(cease, 0.9791468381881714)","(changes, 0.9784608483314514)","(expand, 0.9781211614608765)","(capital, 0.9780576229095459)","(interruption, 0.9780250191688538)","(hinder, 0.9778600335121155)"
disruption,"(could_adversely_affect, 0.9853712916374207)","(conflicts, 0.9840320348739624)","(decline, 0.9800724983215332)","(commercialization_efforts, 0.978864848613739)","(interruption, 0.9785829186439514)","(negatively_affect, 0.9784774780273438)","(raise_substantial_doubt, 0.9771695733070374)","(commercial_prospects, 0.9753139019012451)","(planned, 0.9747026562690735)","(growth, 0.9743182063102722)",...,"(impair, 0.968219518661499)","(investments, 0.9680455327033997)","(reputation, 0.9676132202148438)","(could_harm, 0.9674288630485535)","(impact, 0.967224657535553)","(turn, 0.9671006202697754)","(debt, 0.9670233130455017)","(adversely_affect, 0.9669604301452637)","(reduce, 0.9656952619552612)","(could_materially, 0.9651310443878174)"
lawsuit,"(erisa, 0.9919130206108093)","(maryland, 0.9916706085205078)","(constitutes, 0.9913337826728821)","(obtains, 0.9910709261894226)","(statute, 0.9910081624984741)","(consolidation, 0.9902230501174927)","(statutorily, 0.9900433421134949)","(benchmark, 0.9899604916572571)","(chmp, 0.9899073839187622)","(clinical_benefit, 0.9898038506507874)",...,"(unpredictability, 0.9881022572517395)","(patient_population, 0.9880825877189636)","(official, 0.9879427552223206)","(principles, 0.9877579808235168)","(called, 0.9876043200492859)","(department, 0.9875978231430054)","(effective_january, 0.9875538349151611)","(european, 0.9873859882354736)","(written, 0.9872334003448486)","(abbreviated, 0.9872243404388428)"
litigation,"(substantial_costs, 0.9957731366157532)","(funds, 0.9954367876052856)","(actions, 0.9949911832809448)","(attractive, 0.9948638081550598)","(take, 0.9946092367172241)","(enforce, 0.9943006038665771)","(factors, 0.9939066171646118)","(retain, 0.993747353553772)","(trade_secrets, 0.9936033487319946)","(patents, 0.9936001896858215)",...,"(commercialize, 0.9903440475463867)","(existing, 0.9902036190032959)","(new, 0.990

...

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(september, 0.9956372976303101)","(initiation, 0.9951188564300537)","(jobs_act, 0.9939871430397034)","(earned, 0.9937204122543335)","(eight, 0.9933879971504211)","(valued, 0.9933603405952454)","(initial, 0.9932730793952942)","(second, 0.9930241107940674)","(loss_per_share, 0.9928690195083618)","(there_were, 0.9928016662597656)",...,"(august, 0.9916325807571411)","(estimated_fair, 0.9915130138397217)","(plus, 0.9914568066596985)","(november, 0.9910385608673096)","(trading_days, 0.9910110235214233)","(berkshire_hathaway, 0.9909522533416748)","(purchase_price, 0.9909207820892334)","(annual, 0.9906214475631714)","(quarter_ended, 0.9905802011489868)","(classified_as, 0.9905184507369995)"
injury,"(considering, 0.9979594349861145)","(articles, 0.9979482889175415)","(recent, 0.9976940751075745)","(maximize, 0.9976649880409241)","(live, 0.9969079494476318)","(common, 0.9968677759170532)","(timely, 0.9968211650848389)","(unauthorized_access, 0.996650218963623)","(smaller, 0.9965037107467651)","(contains, 0.9963811635971069)",...,"(indicators, 0.9960606694221497)","(leading, 0.9960517883300781)","(fiduciary, 0.9960397481918335)","(power, 0.9959046840667725)","(street_suite, 0.9958900213241577)","(resolution, 0.9958826303482056)","(representative, 0.9958341717720032)","(civil_penalties, 0.9957932829856873)","(establishing, 0.9957833290100098)","(acting, 0.9957759976387024)"
delay,"(profitability, 0.9949589967727661)","(adverse_impact, 0.9945255517959595)","(their_businesses, 0.9945095777511597)","(inability, 0.9943081140518188)","(proprietary_technology, 0.9932219982147217)","(potential, 0.9931375980377197)","(reasons, 0.9931365251541138)","(collaborators, 0.9929829239845276)","(delays, 0.9926683902740479)","(engage, 0.9926168918609619)",...,"(purchasers, 0.9915764927864075)","(attract, 0.9915623068809509)","(remediate, 0.9913216829299927)","(proprietary_information, 0.9911770820617676)","(successfully, 0.9910597801208496)","(generate_revenue, 0.9909406304359436)","(demands, 0.9907939434051514)","(prevent, 0.990751326084137)","(operate, 0.9906151294708252)","(compete, 0.9905387163162231)"
disruption,"(negatively_affect, 0.9945688247680664)","(could_adversely_affect, 0.9936565160751343)","(prospects, 0.9928359985351562)","(could_harm, 0.9918818473815918)","(capital_resources, 0.9895565509796143)","(will_depend, 0.9878289699554443)","(adversely_impact, 0.9872598052024841)","(harmed, 0.9843966960906982)","(harm, 0.9837308526039124)","(interruption, 0.9835386276245117)",...,"(impede, 0.9778270721435547)","(development_programs, 0.9773444533348083)","(discourage, 0.9764366149902344)","(commercial_prospects, 0.9762957692146301)","(conflicts, 0.9757289886474609)","(negative, 0.9750151634216309)","(incurring_additional, 0.9745650291442871)","(capital, 0.9743931293487549)","(depress, 0.9741729497909546)","(effect_on, 0.9741085171699524)"
lawsuit,"(called, 0.998102068901062)","(controlled_by, 0.9975528717041016)","(application, 0.9974430799484253)","(considers, 0.9974229335784912)","(obtains, 0.9973759651184082)","(adopted, 0.9972711801528931)","(submission, 0.9972180128097534)","(nonfinancial, 0.997174859046936)","(range, 0.9971578121185303)","(c, 0.997123658657074)",...,"(exempt, 0.9963725805282593)","(produced, 0.9963690638542175)","(mechanism, 0.9963164925575256)","(dependent, 0.9963061213493347)","(progression, 0.9962530136108398)","(as_defined, 0.9961973428726196)","(performance_obligations, 0.996190071105957)","(litigation_shall, 0.9961777329444885)","(requirement, 0.9960445761680603)","(adequacy, 0.9959444999694824)"
litigation,"(further, 0.9977561235427856)","(enforce, 0.9977430701255798)","(factors, 0.9971766471862793)","(defend, 0.9969919919967651)","(attractive, 0.9968633055686951)","(commercialize, 0.996755063533783)","(beliefs, 0.9958820939064026)","(licensees, 0.9955289959907532)","(believe, 0.9953787326812744)","(make_it, 0.9952962398529053)",...,"(against_us, 0.99424743

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(modified_mayo, 0.9799950122833252)","(second, 0.9789000153541565)","(august, 0.9783701300621033)","(carrying_value, 0.9768000245094299)","(january, 0.9741740226745605)","(enrolled, 0.9740360379219055)","(aggregate_principal, 0.9738277792930603)","(measured, 0.9738261699676514)","(classified_as, 0.9734594821929932)","(compensation_expense, 0.9728619456291199)",...,"(loan, 0.9678771495819092)","(good_faith_projection, 0.9677876234054565)","(three, 0.967692494392395)","(third, 0.9676545262336731)","(loss_per_share, 0.967523992061615)","(at_least, 0.967351496219635)","(principal_amount, 0.9672240018844604)","(two, 0.9669540524482727)","(amount_accrued, 0.9661848545074463)","(unsecured_term, 0.9657008647918701)"
injury,"(among, 0.9956889748573303)","(air_emissions_wastewater_discharges, 0.995327...","(candidates, 0.9951677322387695)","(small, 0.9951376914978027)","(multi_tenant, 0.9951188564300537)","(formerly_occupied_by, 0.9950711727142334)","(establishing, 0.9949531555175781)","(goals, 0.9946645498275757)","(lifts, 0.994603157043457)","(acute, 0.9945164322853088)",...,"(severity, 0.9933369159698486)","(corporate, 0.9931914210319519)","(institute, 0.9930864572525024)","(presents, 0.9928348660469055)","(sized, 0.992557942867279)","(biomarkers, 0.9925410747528076)","(reserves, 0.992496132850647)","(full_service, 0.9924849271774292)","(rebates, 0.9924706816673279)","(destruction, 0.992440938949585)"
delay,"(inability, 0.9946123361587524)","(materially, 0.9904757142066956)","(market, 0.9886804819107056)","(hinder, 0.9874109625816345)","(limit, 0.9871769547462463)","(be_harmed, 0.9863748550415039)","(could_result, 0.9861598014831543)","(prevent, 0.9860733151435852)","(issued_patents, 0.9858654737472534)","(significantly, 0.9850552082061768)",...,"(retain, 0.9815135598182678)","(litigation, 0.9813798069953918)","(execute, 0.9810224175453186)","(attention, 0.9808543920516968)","(successfully, 0.9808309078216553)","(disruptions, 0.9807384610176086)","(regulatory_approval, 0.980675995349884)","(engage, 0.9805448055267334)","(affect, 0.9803809523582458)","(come, 0.9798535704612732)"
disruption,"(interruption, 0.9850637912750244)","(could_harm, 0.984213650226593)","(business_strategy, 0.9841590523719788)","(acquisitions, 0.9826436042785645)","(could_materially, 0.9817726016044617)","(interruptions, 0.9809640049934387)","(affect, 0.9806970953941345)","(reputation, 0.9798569679260254)","(prospects, 0.9787002205848694)","(s_attention, 0.9766011834144592)",...,"(liquidity, 0.9717406630516052)","(divert_management, 0.9715026021003723)","(materially_adversely_affected, 0.971359789371...","(may_limit, 0.9709087610244751)","(capital_resources, 0.9700685739517212)","(other_factors, 0.9694094657897949)","(could_adversely_affect, 0.9691954255104065)","(applicable_law, 0.9689437747001648)","(a_material_adverse, 0.9688221216201782)","(negative, 0.9678657650947571)"
lawsuit,"(maryland, 0.9957015514373779)","(writing, 0.9954543113708496)","(function, 0.9954338669776917)","(significant_estimates, 0.9949730038642883)","(utilizing, 0.9938592314720154)","(forming, 0.9937465786933899)","(taking_into_account, 0.9937366843223572)","(qualitative_factors, 0.993676483631134)","(called, 0.9935619831085205)","(determinations, 0.9934759736061096)",...,"(procedure, 0.9921538829803467)","(treating, 0.9921138882637024)","(medicines, 0.9920528531074524)","(stable, 0.9919736981391907)","(equals, 0.9919536113739014)","(parallel, 0.9916907548904419)","(nestl, 0.9916669130325317)","(independence, 0.9916526079177856)","(overall_presentation, 0.9916028380393982)","(spore, 0.9915850758552551)"
litigation,"(retain, 0.9942134618759155)","(factors, 0.9935693740844727)","(substantial_costs, 0.9935640692710876)","(devote, 0.9934536218643188)","(conduct, 0.9924049377441406)","(developments, 0.9914525151252747)","(collaborators, 0.9904759526252747)","(undertake, 0.990414559841156)","(investors, 0.9901877641677856)","(actua

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(upfront, 0.9909819960594177)","(measured, 0.9825847148895264)","(installments, 0.9822729229927063)","(weighted_average, 0.9794208407402039)","(environmental_reserve, 0.9787719249725342)","(ending, 0.978529691696167)","(january, 0.9773054122924805)","(years, 0.973896861076355)","(total, 0.9734298586845398)","(endoscopic, 0.9734042882919312)",...,"(unsecured_term, 0.9669782519340515)","(second, 0.9669116735458374)","(measurement_period, 0.9656227231025696)","(feature, 0.9654245972633362)","(amortized, 0.9653995633125305)","(achievement, 0.9647920727729797)","(tolerated, 0.9646962285041809)","(initiated, 0.9643349647521973)","(initial_funding, 0.9631305932998657)","(recognized_over, 0.963036298751831)"
injury,"(counties, 0.9965102672576904)","(allocations, 0.9955883026123047)","(high, 0.9955271482467651)","(responsibilities, 0.9949996471405029)","(money_market, 0.9949631094932556)","(executive_compensation, 0.9949358701705933)","(concentrations, 0.9948567748069763)","(leading, 0.9946419596672058)","(deposit, 0.994583249092102)","(led, 0.9944409728050232)",...,"(common, 0.993488073348999)","(rapid, 0.9934033751487732)","(emerging_growth_company, 0.9932787418365479)","(innovation, 0.9932231903076172)","(davidson, 0.9931902289390564)","(seeks, 0.9930554628372192)","(biotechnology, 0.9930455088615417)","(banking_offices, 0.9929969906806946)","(acute, 0.9929876327514648)","(cloud_based, 0.9929274320602417)"
delay,"(prevent, 0.9955501556396484)","(limit, 0.995344877243042)","(regulatory_approval, 0.9926046133041382)","(some, 0.9923473596572876)","(commercialize_products, 0.9921780824661255)","(cause, 0.9918356537818909)","(our_licensors, 0.9912183880805969)","(ways, 0.9897593259811401)","(market, 0.9896758794784546)","(disruptions, 0.9890512228012085)",...,"(adversely_impacted, 0.9847949147224426)","(potential, 0.9844274520874023)","(successfully, 0.9842048287391663)","(conflicts, 0.9842015504837036)","(reasons, 0.9830676913261414)","(renew_their, 0.9828197956085205)","(competitors, 0.98220294713974)","(factors, 0.981939435005188)","(compete, 0.9815990328788757)","(adversely_affect, 0.9813598394393921)"
disruption,"(reputation, 0.9924135804176331)","(could_harm, 0.990497350692749)","(material_weaknesses, 0.9901762008666992)","(harm, 0.9885358810424805)","(could_materially, 0.9875655174255371)","(attention, 0.98388671875)","(divert_management, 0.9812098145484924)","(affect, 0.981033980846405)","(development_programs, 0.9795956015586853)","(expand, 0.9795016050338745)",...,"(demand, 0.9699639081954956)","(be_adversely_affected, 0.9696803092956543)","(adversely_affect, 0.9694300889968872)","(experience, 0.968936562538147)","(normal_course, 0.9687840938568115)","(restrict, 0.9685035347938538)","(may_limit, 0.9682852029800415)","(liquidity, 0.9672925472259521)","(casualty_event, 0.9668609499931335)","(changes, 0.9660707712173462)"
lawsuit,"(genetically, 0.9950026273727417)","(requires, 0.994713544845581)","(document, 0.9945256114006042)","(body, 0.9943992495536804)","(colonic_microbiome, 0.994377613067627)","(statute, 0.9942849278450012)","(systemically_administered, 0.9942158460617065)","(pensacola, 0.9940502643585205)","(antigens, 0.9935234785079956)","(g, 0.9935188889503479)",...,"(newton, 0.9929081201553345)","(determinations, 0.9928884506225586)","(medicines, 0.992875337600708)","(antigen, 0.9928672313690186)","(locally, 0.9928364157676697)","(msas, 0.9928156733512878)","(consumer_protection_act, 0.9927759170532227)","(new_york, 0.9927699565887451)","(residual, 0.9926949739456177)","(colitis, 0.9926737546920776)"
litigation,"(could_result, 0.9956987500190735)","(proprietary_information, 0.9943821430206299)","(substantial_costs, 0.9934120178222656)","(competitive_position, 0.9932279586791992)","(opportunities, 0.9930356740951538)","(compete, 0.9925371408462524)","(pressures, 0.9923317432403564)","(proprietary_rights, 0.9920865297317505)","(efforts, 0.9920340776443481)","(da

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(upfront, 0.9726945161819458)","(straight_line_basis, 0.9725874066352844)","(trading_days, 0.9706842303276062)","(january, 0.9697691798210144)","(ending, 0.9680372476577759)","(weighted_average, 0.9671953916549683)","(evaluable, 0.9662237167358398)","(fiscal_quarter, 0.9657717347145081)","(unsecured_term, 0.9630498886108398)","(valued, 0.9616714119911194)",...,"(second, 0.957270622253418)","(endoscopic, 0.9572080373764038)","(environmental_reserve, 0.9565134048461914)","(settled, 0.9560384154319763)","(total, 0.9551441669464111)","(carrying_value, 0.9540386199951172)","(last, 0.9535338878631592)","(over, 0.9531353712081909)","(amortized, 0.9528223872184753)","(aggregate_principal, 0.9526044726371765)"
injury,"(recycling, 0.9969642162322998)","(supplementary, 0.9964688420295715)","(inventory, 0.996315598487854)","(ta_s, 0.9955956339836121)","(directly, 0.9955242872238159)","(prosecution, 0.9955060482025146)","(see, 0.995415449142456)","(disclosures, 0.995029628276825)","(increasing, 0.9949009418487549)","(stages, 0.9948830008506775)",...,"(sought, 0.9940265417098999)","(shopping, 0.9939131736755371)","(preclinical, 0.9938613772392273)","(resource, 0.9938458800315857)","(clinical_studies, 0.9938104152679443)","(plans, 0.9937676191329956)","(adr, 0.9937164187431335)","(paying_dividends, 0.9936962723731995)","(proprietary, 0.993590235710144)","(acts, 0.9935694336891174)"
delay,"(limit, 0.9936636686325073)","(cause, 0.9930590391159058)","(be_adversely_affected, 0.9918897747993469)","(inability, 0.9909266829490662)","(may_suffer, 0.9908849000930786)","(successfully, 0.9897593855857849)","(diminish, 0.9897576570510864)","(delays, 0.9881945252418518)","(impair, 0.9877744913101196)","(profitability, 0.9877683520317078)",...,"(meet, 0.9852473735809326)","(prevent, 0.9849762916564941)","(renew_their, 0.9848852157592773)","(commercialization_efforts, 0.9840467572212219)","(operate, 0.9838300943374634)","(an_adverse, 0.9833922982215881)","(sustain, 0.9829601645469666)","(jeopardize, 0.9828381538391113)","(attention, 0.9827338457107544)","(substantial_costs, 0.9827027320861816)"
disruption,"(s_attention, 0.9910362362861633)","(adversely_impact, 0.9889240264892578)","(raise_substantial_doubt, 0.9885619282722473)","(confidence, 0.9871144890785217)","(prospects, 0.9834660291671753)","(interruption, 0.9833337068557739)","(trading_volume, 0.9829540252685547)","(could_materially, 0.9828732013702393)","(reputation, 0.98273766040802)","(be_harmed, 0.9806594252586365)",...,"(distributors, 0.9737096428871155)","(effect_on, 0.9730345606803894)","(negatively_impacted, 0.9729311466217041)","(attract, 0.9715890884399414)","(adversely_affect, 0.9711023569107056)","(important_factors, 0.9700452089309692)","(significantly, 0.9699013233184814)","(impair, 0.9684772491455078)","(commercialization_efforts, 0.9682003259658813)","(significant, 0.9664344787597656)"
lawsuit,"(considers, 0.9971445202827454)","(probability_weighted, 0.9965369701385498)","(requires, 0.9951387643814087)","(taking_into_account, 0.9947549700737)","(authority, 0.9946381449699402)","(cash_flows_undiscounted, 0.994568407535553)","(standardized, 0.9933378100395203)","(deciding, 0.9931126236915588)","(judgment, 0.9931043386459351)","(official, 0.9930995106697083)",...,"(makes, 0.9917405843734741)","(supplemental, 0.9915110468864441)","(active, 0.9913361668586731)","(very, 0.9912847280502319)","(statistical, 0.9912499189376831)","(approves, 0.9912083745002747)","(budget, 0.9910345077514648)","(took, 0.9909220337867737)","(qualitative_factors, 0.9908376932144165)","(ci, 0.9908340573310852)"
litigation,"(differ_materially_from, 0.9964288473129272)","(factors, 0.9959637522697449)","(their_businesses, 0.9956492781639099)","(execute, 0.9952313899993896)","(devote, 0.995111346244812)","(make_it, 0.9950923323631287)","(prospective, 0.9947326183319092)","(developments, 0.9947105646133423)","(substantial_costs, 0.9940104484558105)","(potent

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
strike,"(third, 0.9915273189544678)","(compensation_expense, 0.991283118724823)","(per_common, 0.9902939796447754)","(second, 0.9900634288787842)","(guarantor_entered_into, 0.9900025129318237)","(august, 0.9899756908416748)","(june, 0.9892956018447876)","(trial, 0.9889345765113831)","(interim_periods, 0.9881426095962524)","(initiation, 0.9880145192146301)",...,"(valued, 0.9854888916015625)","(mortgage_loans, 0.9853769540786743)","(three, 0.9853448867797852)","(entered_into, 0.984883189201355)","(july, 0.984840989112854)","(measured, 0.9847002029418945)","(categorized, 0.9845777750015259)","(vesting_period, 0.9843989610671997)","(september, 0.9841392040252686)","(november, 0.9840851426124573)"
injury,"(optimize, 0.998120903968811)","(fields, 0.9979519844055176)","(retenanting, 0.9979355335235596)","(increasing, 0.9979045987129211)","(corporate, 0.9977221488952637)","(likewise, 0.9976460933685303)","(contractors, 0.9976238012313843)","(shopping_centers, 0.9973428249359131)","(fermentation, 0.9972884654998779)","(recycling, 0.9971587061882019)",...,"(resulting_from, 0.9965324401855469)","(diversified, 0.996509313583374)","(validity_enforceability, 0.9965016841888428)","(interference, 0.9964191913604736)","(generation, 0.9964176416397095)","(predominately, 0.9960973858833313)","(reimbursement_rates, 0.9960971474647522)","(prosecution, 0.9960264563560486)","(robust, 0.9960013628005981)","(timelines, 0.9960002899169922)"
delay,"(cause, 0.9970383644104004)","(limit, 0.9942458271980286)","(be_adversely_affected, 0.9937676191329956)","(reasons, 0.9931479692459106)","(prevent, 0.99141526222229)","(disruptions, 0.9913787245750427)","(execute, 0.9909643530845642)","(negatively_impact, 0.9909359812736511)","(carry_out, 0.9908848404884338)","(market, 0.9904192686080933)",...,"(take, 0.9868993759155273)","(their_businesses, 0.9867783784866333)","(important_factors, 0.9866311550140381)","(substantial, 0.9866271615028381)","(materially_adversely_affected, 0.986566781997...","(impair, 0.9865365028381348)","(substantial_costs, 0.9859918355941772)","(compete, 0.9852688312530518)","(successfully, 0.9846694469451904)","(some, 0.9846080541610718)"
disruption,"(business_strategy, 0.9877769947052002)","(prospects, 0.9861994981765747)","(confidence, 0.9856528043746948)","(diversion, 0.9855685234069824)","(could_harm, 0.9853201508522034)","(s_attention, 0.9848312735557556)","(could_materially, 0.982325553894043)","(other_factors, 0.9789247512817383)","(success_depends, 0.9781116247177124)","(breadth, 0.9767628908157349)",...,"(impede, 0.9698618650436401)","(indicative, 0.9692440629005432)","(risks_uncertainties, 0.9691702127456665)","(attention, 0.9686431884765625)","(dealings, 0.9684213399887085)","(competitive_position, 0.9680938720703125)","(reputation, 0.9678739905357361)","(depress, 0.9671924114227295)","(development_programs, 0.9666990041732788)","(commercialization_efforts, 0.9662328958511353)"
lawsuit,"(factors_including, 0.9977197647094727)","(assessing, 0.9967727661132812)","(reviews, 0.9966611862182617)","(model, 0.9965855479240417)","(significant_estimates, 0.9965283870697021)","(residual, 0.9964931607246399)","(locally, 0.996310830116272)","(assigned, 0.9960223436355591)","(contractual_terms, 0.9958837032318115)","(gives, 0.9958556890487671)",...,"(procedures_included, 0.9950761795043945)","(budget, 0.9950700402259827)","(amended_complaint_relating, 0.9950248003005981)","(organizational, 0.9950248003005981)","(innovation, 0.9949811697006226)","(l_p, 0.9948878288269043)","(litigation_unjust_enrichment, 0.9948840737342...","(flagship_pioneering, 0.9948534965515137)","(commonly, 0.9947555065155029)","(defines, 0.9947305917739868)"
litigation,"(retain, 0.9976339340209961)","(aspects, 0.9970748424530029)","(prospective, 0.9968007802963257)","(make_it, 0.9967823028564453)","(efforts, 0.995910108089447)","(conduct, 0.9955432415008545)","(potential, 0.9953303337097168)","(matters, 0.99513447284

In [ ]:
# less co terms
gs2 = gs(tri_sentences, min_count = [10], context_size = [5, 10, 15], vector_size = [50, 100, 150, 200])

2022-04-04 13:44:14,404 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2022-04-04 13:44:14,410 : INFO : collecting all words and their counts
2022-04-04 13:44:14,415 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-04 13:44:14,559 : INFO : PROGRESS: at sentence #10000, processed 273729 words, keeping 9881 word types
2022-04-04 13:44:14,574 : INFO : collected 10727 word types from a corpus of 326578 raw words and 11949 sentences
2022-04-04 13:44:14,576 : INFO : Loading a fresh vocabulary
2022-04-04 13:44:14,591 : INFO : effective_min_count=10 retains 3579 unique words (33% of original 10727, drops 7148)
2022-04-04 13:44:14,592 : INFO : effective_min_count=10 leaves 300934 word corpus (92% of original 326578, drops 25644)


Training model...


2022-04-04 13:44:14,606 : INFO : deleting the raw counts dictionary of 10727 items
2022-04-04 13:44:14,609 : INFO : sample=0.001 downsamples 37 most-common words
2022-04-04 13:44:14,612 : INFO : downsampling leaves estimated 210046 word corpus (69.8% of prior 300934)
2022-04-04 13:44:14,625 : INFO : estimated required memory for 3579 words and 50 dimensions: 3221100 bytes
2022-04-04 13:44:14,626 : INFO : resetting layer weights
2022-04-04 13:44:15,312 : INFO : training model with 4 workers on 3579 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-04-04 13:44:15,658 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:44:15,667 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:44:15,680 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:44:15,684 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:44:15,687 : INFO : EPOCH - 1 : training on

Training model... 50features_10minwords_5context_trigram


2022-04-04 13:44:17,980 : INFO : training model with 4 workers on 3579 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-04-04 13:44:18,340 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:44:18,354 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:44:18,368 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:44:18,373 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:44:18,375 : INFO : EPOCH - 1 : training on 326578 raw words (210059 effective words) took 0.4s, 547791 effective words/s
2022-04-04 13:44:18,721 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:44:18,731 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:44:18,739 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:44:18,743 : INFO : worker thread finished; awaiting finish of 0 mo

...

2022-04-04 13:45:20,152 : INFO : training model with 4 workers on 3579 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:45:20,666 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:20,673 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:20,679 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:20,688 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:45:20,689 : INFO : EPOCH - 1 : training on 326578 raw words (209741 effective words) took 0.5s, 400255 effective words/s
2022-04-04 13:45:21,184 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:21,193 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:21,205 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:21,217 : INFO : worker thread finished; awaiting finish of 0 

Training model...


2022-04-04 13:45:23,831 : INFO : training model with 4 workers on 3579 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:45:24,270 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:24,272 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:24,273 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:24,276 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:45:24,277 : INFO : EPOCH - 1 : training on 326578 raw words (209968 effective words) took 0.4s, 481223 effective words/s
2022-04-04 13:45:24,717 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:24,734 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:24,742 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:24,744 : INFO : worker thread finished; awaiting finish of 0 

Training model... 200features_10minwords_15context_trigram


2022-04-04 13:45:27,023 : INFO : training model with 4 workers on 3579 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2022-04-04 13:45:27,564 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:27,576 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:27,598 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:27,611 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-04 13:45:27,612 : INFO : EPOCH - 1 : training on 326578 raw words (209965 effective words) took 0.6s, 367617 effective words/s
2022-04-04 13:45:28,185 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-04 13:45:28,204 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-04 13:45:28,210 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-04 13:45:28,218 : INFO : worker thread finished; awaiting finish of 0 